In [19]:
#Purpose of the notebook is to create 2 different M2C datasets, one for disease and one for phenotype.

import os
import configparser

import pandas as pd

In [25]:
DATASET = '../dataset'
CONFIGURATIONS = '../configurations'

CF = configparser.ConfigParser()
CF.read(os.path.join(CONFIGURATIONS, 'data.config'))

/home/akshita/anaconda3/envs/scripps/lib/python2.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  """


['../configurations/data.config']

In [27]:
def __load_path(path):
    with open(path) as file_handle:
        contents = file_handle.read()
        lines = contents.split('\n')
        annos = []
        for line in lines:
            if line:
                annos.append(line.split(' '))
        data = pd.DataFrame(annos)
    return data

def __get_path(dataset, mode):
    assert dataset in ['disease', 'phenotype', 'm2c', 'm2c_citizen_disease', \
                       'm2c_citizen_phenotype', 'm2c_expert_disease', 'm2c_expert_phenotype']
    path = os.path.join(CF.get(dataset, 'path'), CF.get(dataset, mode))
    return os.path.join(DATASET, path)

def load_dataset(dataset='disease', mode='train'):
    data = __load_path(__get_path(dataset, mode))
    #data = pd.read_csv(os.path.join(DATASET, path), sep=' ', header=None) #misses some rows for some reason.
    return data

def get_statistics(data, true_label='I-Disease'):
    num_train = len(data)
    num_true = len(data.loc[data[3] == true_label])
    perc = (float(num_true)/float(num_train)) * 100
    #print 'Number of training samples: {}'.format(num_train)
    print 'Percent of {}: {}%'.format(true_label, perc)

In [33]:
for i in range(1, 6):
    m2c = load_dataset('m2c', 'files{}'.format(i))
    disease_labels = ['I-Disease', 'I-Disease_Pheno', 'I-Disease_Phen']
    
    mask = m2c[3].isin(disease_labels)
    m2c.loc[mask, 3] = 'I-Disease'
    
    mask = m2c[3] == 'I-Phenotype'
    m2c.loc[mask, 3] = 'O'
    
    m2c.to_csv(__get_path('m2c_citizen_disease', 'files{}'.format(i)), sep=' ', index=False, header=False)
    
    
    m2c = load_dataset('m2c', 'files{}'.format(i))
    
    phenotype_labels = ['I-Phenotype', 'I-Disease_Pheno', 'I-Disease_Phen']
    
    mask = m2c[3].isin(phenotype_labels)
    m2c.loc[mask, 3] = 'I-Phenotype'
    
    mask = m2c[3] == 'I-Disease'
    m2c.loc[mask, 3] = 'O'
    
    m2c.to_csv(__get_path('m2c_citizen_phenotype', 'files{}'.format(i)), sep=' ', index=False, header=False)